<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Setup---imports" data-toc-modified-id="Setup---imports-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Setup - imports</a></span></li><li><span><a href="#Setup---django" data-toc-modified-id="Setup---django-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Setup - django</a></span></li><li><span><a href="#Setup---file-paths" data-toc-modified-id="Setup---file-paths-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Setup - file paths</a></span></li></ul></li><li><span><a href="#Work" data-toc-modified-id="Work-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Work</a></span><ul class="toc-item"><li><span><a href="#Work---coding-user-and-work-log" data-toc-modified-id="Work---coding-user-and-work-log-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Work - coding user and work log</a></span><ul class="toc-item"><li><span><a href="#Work----coding-user" data-toc-modified-id="Work----coding-user-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Work  - coding user</a></span></li><li><span><a href="#Work---create-work-log" data-toc-modified-id="Work---create-work-log-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Work - create work log</a></span></li></ul></li><li><span><a href="#Work---load-JSON-file" data-toc-modified-id="Work---load-JSON-file-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Work - load JSON file</a></span></li><li><span><a href="#Work---process-JSON" data-toc-modified-id="Work---process-JSON-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Work - process JSON</a></span></li></ul></li><li><span><a href="#Evaluate" data-toc-modified-id="Evaluate-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Evaluate</a></span></li></ul></div>

# Setup

- Back to [Table of Contents](#Table-of-Contents)

## Setup - imports

- Back to [Table of Contents](#Table-of-Contents)

In [ ]:
import calendar
import datetime
import json
import six

## Setup - django

- Back to [Table of Contents](#Table-of-Contents)

Specify path to `django_init.py`, then run it.

In [ ]:
django_init_folder = ".."
django_init_file = "django_init.py"
django_init_path = django_init_file

# got a folder?
if ( ( django_init_folder is not None ) and ( django_init_folder != "" ) ):
    
    # prepend folder to path.
    django_init_path = "{}/{}".format( django_init_folder, django_init_path )
    
#-- END check to see if folder --#

In [ ]:
%run -i $django_init_path

In [ ]:
# imports
from context.models import Work_Log
from django.contrib.auth.models import User
from sourcenet.article_coding.manual_coding.manual_article_coder import ManualArticleCoder
from sourcenet.models import Article
from sourcenet.models import Article_Data
from context_data.models import DataSet
from context_data.models import DataReference
from context_data.models import DataReferenceMention
from context_data.models import DataReferenceContext

## Setup - file paths

- Back to [Table of Contents](#Table-of-Contents)

In [ ]:
# JSON file path
json_file_name = "rcc-phase2-data_set_data.json"
json_file_path = "./{}".format( json_file_name )

# output file
output_folder_path = "."

# Work

- Back to [Table of Contents](#Table-of-Contents)

Steps:

- 1) Set up work log and user.
- 2) load JSON file into memory.
- 3) loop over objects in list.
- 4) for each: retrieve data, store it in appropriate place in database.



## Work - coding user and work log

- Back to [Table of Contents](#Table-of-Contents)

### Work  - coding user

- Back to [Table of Contents](#Table-of-Contents)

Check to see if coding user for current competition group already exists, if not make one.

In [ ]:
# declare variables
coding_user = None
username = "elevitskaya"
first_name = "Ekaterina"
last_name = "Levitskaya"
email_address = "el2727@nyu.edu"

# work variables
temp_user = None
temp_password = None

# use a try to detect if user exists.
try:

    # get user
    temp_user = User.objects.get( username = username )

    print( "Found user {}".format( temp_user ) )

except:

    # set password to current time stamp.
    temp_password = datetime.datetime.utcnow()
    temp_password = calendar.timegm( temp_password.timetuple() )
    temp_password = str( temp_password )

    # create user with username, password, no email.
    temp_user = User.objects.create_user( username, email_address, temp_password )

    # add information to user.
    temp_user.first_name = first_name
    temp_user.last_name = last_name

    # save user.
    temp_user.save()
    
    print( "Created user {}, password: {}".format( temp_user, temp_password ) )

#-- END try/except for looking up automated user. --#

# store user
coding_user = temp_user


### Work - create work log

- Back to [Table of Contents](#Table-of-Contents)

Now create a work log entry to associate with all of the data created as part of loading the team's submission.

In [ ]:
# declare variables
work_log_entry = None
work_log_slug = "DataReference_test_load"
work_log_name = work_log_slug
work_log_user = coding_user
work_log_tag_list = None

# work variables
temp_work_log = None
work_log_qs = None

# tags to add
work_log_tag_list = []
work_log_tag_list.append( work_log_slug )

# use a try to detect if user exists.
try:

    # get work log entry
    temp_work_log = Work_Log.objects.get( slug = work_log_slug )

    print( "Found work log: {}".format( temp_work_log ) )

except:

    # create work log record.
    temp_work_log = Work_Log()

    # add details
    temp_work_log.slug = work_log_slug
    temp_work_log.name = work_log_name
    temp_work_log.worker = coding_user
    
    # save user.
    temp_work_log.save()

    # manage tags:
    for tag_name in work_log_tag_list:
        
        # add tag
        temp_work_log.tags.add( tag_name )

    #-- END loop over tags. --#
        
    print( "Created work log record {}".format( temp_work_log ) )

#-- END try/except for looking up automated user. --#

# store user
work_log_entry = temp_work_log


## Work - load JSON file

- Back to [Table of Contents](#Table-of-Contents)

load the JSON file into memory.

In [ ]:
# load data reference info from JSON.
path_to_json = json_file_path

# open file and parse into JSON.
json_file = None
json_data = None

# open file
with open( path_to_json ) as json_file:

    # load JSON data
    json_data = json.load( json_file )
    
#-- END with open() as json_file --#

print( "loaded JSON from {} @ {}".format( path_to_json, datetime.datetime.now() ) )

## Work - process JSON

- Back to [Table of Contents](#Table-of-Contents)

Loop over items in JSON, processing all that are marked as "include" = 1.

In [ ]:
def get_citation_type_for_mention_type( mention_type_IN, is_project_data_IN ):
    
    # return reference
    value_OUT = None
    
    # declare variables
    mention_type = None
    is_project_data = None
    citation_type = None
    
    # init
    mention_type = mention_type_IN
    is_project_data = is_project_data_IN
    
    # got a mention type?
    if ( mention_type is not None ):
    
        # Translate to citation type
        if ( mention_type == "cited" ):

            # "cited" = CITATION_TYPE_MENTION
            citation_type = DataReference.CITATION_TYPE_MENTION

        elif ( mention_type == "analysis" ):

            # "analysis" = CITATION_TYPE_MENTION
            citation_type = DataReference.CITATION_TYPE_ANALYSIS

        elif ( mention_type == "statistic" ):

            # "statistic" = CITATION_TYPE_STATISTIC
            citation_type = DataReference.CITATION_TYPE_STATISTIC

        else:

            # not one of the mention types we are focusing on, so None
            citation_type = DataReference.CITATION_TYPE_DEFAULT

        #-- end translate from mention_type to citation_type --#

        # check to see if project-specific data.  If so, use citation type CITATION_TYPE_PROJECT.
        if ( ( is_project_data is not None ) and ( is_project_data == "yes" ) ):

            # project-specific data.
            citation_type = DataReference.CITATION_TYPE_PROJECT

        #-- END check to see if project-specific data. --#

    else:
        
        # ERROR - no mention type, return Default
        citation_type = DataReference.CITATION_TYPE_DEFAULT
        
    #-- END check to see if mention type --#
    
    value_OUT = citation_type
    
    return value_OUT

#-- END function get_citation_type_for_mention_type() --#

In [ ]:
# declare variables
reference_json = None

# declare variables - values in JSON
include_flag = None  # "include"
publication_id = None  # "publicationId"
related_article = None
mention_type = None  # "mentiontypeCitedanalysisstatistic" - "cited", "statistic", "analysis"
citation_type = None
data_title = None  # "titleIfPresent"
data_set_ids = None  # "dataSetId"
data_set_id_list = None
data_set_id = None
related_data_set_list = None
is_project_data = None  # "projectData"
key_terms = None  # "keyTerms"
ref_id = None  # "refid"
index = None  # "index"
evaluator = None  # "evaluator"
evaluator_user = None  # "evaluatoruser"
data_set_number = None  # "data_set_number"
mention_list = None  # "mention#"
context_list = None  # "context#"
current_notes = None

# declare variables - processing variables
prop_name_list = None
prop_name = None
prop_value = None
do_save = True

# declare variables - add to database.
article_coding_helper = None
article_data_result = None
article_data = None
data_reference = None
mention_text = None
data_mention = None
context_text = None
data_context = None

# get instance of ManualArticleCoder
article_coding_helper = ManualArticleCoder()

# loop over references.
for reference_json in json_data:
    
    # include flag
    include_flag = reference_json.get( "include", 0 )
    
    # included?
    if ( ( include_flag is not None ) and ( include_flag > 0 ) ):

        # yes - make sure there is a publication ID.
        publication_id = reference_json.get( "publicationId", None )
        if ( ( publication_id is not None ) and ( publication_id > 0 ) ):
            
            # got a publication.  Retrieve it.
            related_article = Article.objects.get( id = publication_id )
            
            # retrieve other reference details
            mention_type = reference_json.get( "mentiontypeCitedanalysisstatistic", DataReference.CITATION_TYPE_DEFAULT )
            data_title = reference_json.get( "titleIfPresent", None )
            data_set_ids = reference_json.get( "dataSetId", None )
            is_project_data = reference_json.get( "projectData", None )
            key_terms = reference_json.get( "keyTerms", None )

            # Translate to citation type
            citation_type = get_citation_type_for_mention_type( mention_type, is_project_data )
            
            # multiple data sets?
            if ( ( data_set_ids is not None ) and ( data_set_ids != "" ) ):

                # we have something...
                try:

                    data_set_id_list = data_set_ids.split( "," )

                except:

                    # integer - just put it in the list alone.
                    data_set_id_list = [ str( data_set_ids ) ]

                #-- END try...except --#

                # do we have related data sets?
                related_data_set_list = []
                for data_set_id in data_set_id_list:

                    # not empty, and not "NA"?
                    data_set_id = data_set_id.strip()
                    if ( ( data_set_id is not None ) and ( data_set_id != "" ) and ( data_set_id != "NA" ) ):

                        # convert to integer
                        data_set_id = int( data_set_id )
                        if ( data_set_id > 0 ):
                            
                            # retrieve data set
                            related_data_set = DataSet.objects.get( id = data_set_id )
                            
                            # add to list
                            related_data_set_list.append( related_data_set )
                            
                        #-- END check to see if data set ID is number greater than 0 --#
                        
                    #-- END check to see if data set ID empty --#
                    
                #-- END loop over data set IDs. --#

            #-- END check to see if data set id values. --#

            # to retrieve mentions and contexts, need to get list of keys in object.
            prop_name_list = list( six.viewkeys( reference_json ) )
            mention_list = []  # "mention#"
            context_list = []  # "context#"
            for prop_name in prop_name_list:
                
                # does prop name start with "context"?
                if ( prop_name.startswith( "context" ) == True ):
                    
                    # get context and add to list.
                    prop_value = reference_json.get( prop_name, None )
                    if ( ( prop_value is not None ) and ( prop_value != "" ) ):

                        context_list.append( prop_value )
                                             
                    #-- END check to see if non-empty value. --#
                
                #-- END check to see if context --#
            
                # does prop name start with "mention" and not named
                #     "mentiontypeCitedanalysisstatistic"?
                if ( ( prop_name.startswith( "mention" ) == True )
                    and ( prop_name != "mentiontypeCitedanalysisstatistic" ) ):
                    
                    # get context and add to list.
                    prop_value = reference_json.get( prop_name, None )
                    if ( ( prop_value is not None ) and ( prop_value != "" ) ):

                        mention_list.append( prop_value )
                                             
                    #-- END check to see if non-empty value. --#
                                
                #-- END check to see if context --#
            
            #-- END loop over property names --#
            
            # Store in database.
            
            # get Article_Data for user and publication.
            article_data_result = article_coding_helper.lookup_article_data( related_article,
                                                                             coding_user,
                                                                             work_log_IN = work_log_entry )
            article_data = article_data_result.get( ManualArticleCoder.PROP_ARTICLE_DATA )
            if ( article_data is not None ):
            
                # TODO - check for existing.
            
                # ==> make a data reference
                data_reference = DataReference()

                # set values
                data_reference.article = related_article
                data_reference.title = data_title
                data_reference.citation_type = citation_type
                data_reference.key_terms = key_terms
                data_reference.work_log = work_log_entry

                # set values - bookkeeping
                data_reference.article_data = article_data
                data_reference.coder = coding_user

                # store raw JSON in notes.
                data_reference.notes = json.dumps( reference_json, sort_keys = True, indent = 4 )

                # save it.
                if ( do_save == True ):
                    data_reference.save()

                    # related data sets
                    if ( len( related_data_set_list ) > 0 ):

                        # loop over data sets
                        for related_data_set in related_data_set_list:

                            # add to related.
                            data_reference.related_data_sets.add( related_data_set )

                        #-- END loop over related data sets. --#

                    #-- END check to see if related data sets --#
                
                #-- END do_save? --#

                # ==> data mentions
                for mention_text in mention_list:
                    
                    # TODO - check for existing.
                    
                    # create data_mention
                    data_mention = DataReferenceMention()
                    
                    # store values
                    data_mention.data_reference = data_reference
                    data_mention.value = mention_text
                    
                    # if you needed or wanted to, could do processing to locate
                    #     in canonical text here.
                    
                    # save.
                    if ( do_save == True ):
                        data_mention.save()
                    #-- END do save? --#
                    
                #-- END loop over mentions --#

                # ==> data context
                for context_text in context_list:
                    
                    # TODO - check for existing.
                    
                    # create data_citation
                    data_context = DataReferenceContext()
                    
                    # store values
                    data_context.data_reference = data_reference
                    data_context.value = context_text
                    
                    # if you needed or wanted to, could do processing to locate
                    #     in canonical text here.
                    
                    # save.
                    if ( do_save == True ):
                        data_context.save()
                    #-- END do_save? --#
                    
                #-- END loop over contexts --#                    
                
            else:
                
                # no Article_Data returned - hmmm...
                print( "ERROR - No Article data returned for article {}; user {}; work log {}.  Result: {}".format( related_article, coding_user, work_log_entry, article_data_result ) )
                
            #-- END check to see if Article_Data instance. --#

        #-- END check for publication ID --#
        
    #-- END check to see if publication ID --#
    
#-- END loop over data references --#

# Evaluate

- Back to [Table of Contents](#Table-of-Contents)

Now, loop over the records for this work log and create JSON, so we can see how it looks.

In [ ]:
# Create QuerySet of DataReference instances
data_reference_qs = None

# get data references that refer to our work log entry
data_reference_qs = DataReference.objects.filter( work_log = work_log_entry )

# output count
print( data_reference_qs.count() )

In [ ]:
# declare variables
article_data_qs = None
article_data = None
data_reference_qs = None
data_reference_count = None
data_reference_counter = None
data_reference = None
related_article = None
related_article_id = None
work_log = None
work_log_label = None
update_counter = None

# declare variables - create reference JSON data
reference_list = None
reference_dict = None

# declare variables - data set list
data_set_qs = None
data_set_id_list = None
data_set = None

# declare variables - mention list
mention_qs = None
mention_list = None
mention = None
mention_dict = None

# declare variables - context list
context_qs = None
context_list = None

# get Article_Data for work log.
article_data_qs = Article_Data.objects.filter( work_log = work_log_entry )
print( article_data_qs.count() )

# loop over Article_Data
data_reference_count = 0
data_reference_counter = 0
update_counter = 0
reference_list = []
for article_data in article_data_qs:
    
    # retrieve data references
    data_reference_qs = article_data.datareference_set.all()
    data_reference_count = data_reference_qs.count()
    print( "==> AD {} reference count: {}".format( article_data.id, data_reference_count ) )
    data_reference_counter += data_reference_count
    
    # loop over data references.
    for data_reference in data_reference_qs:
        
        # add work log.
        if ( data_reference.work_log is None ):
            
            # no work log.  Add one.
            data_reference.work_log = work_log_entry
            data_reference.save()
            update_counter += 1
            
        #-- END check if work log is set. --#
        
        # build JSON.
        reference_dict = {}
        
        # add fields from reference
        
        # article ID
        related_article = data_reference.article
        related_article_id = related_article.id
        reference_dict[ "article_id" ] = related_article_id
        
        # title
        reference_dict[ "title" ] = data_reference.title
        
        # citation type
        reference_dict[ "citation_type" ] = data_reference.citation_type
        
        # key terms
        reference_dict[ "key_terms" ] = data_reference.key_terms
        
        # work log label
        work_log = data_reference.work_log
        work_log_label = work_log.slug
        reference_dict[ "work_log_label" ] = work_log_label
        
        # data sets
        data_set_id_list = []
        for data_set in data_reference.related_data_sets.all():
            
            # add ID to list.
            data_set_id_list.append( data_set.id )
            
        #-- END loop over related data sets. --#
        reference_dict[ "related_data_set_id_list" ] = data_set_id_list        
        
        # mentions
        mention_list = []
        mention_qs = data_reference.datareferencemention_set.all()
        for mention in mention_qs:
            
            # add value, start index, and occurrence number to list.
            mention_dict = {}
            mention_dict[ "value" ] = mention.value
            mention_dict[ "start_index" ] = mention.start_index
            mention_dict[ "occurrence_number" ] = mention.occurrence_number
            mention_list.append( mention_dict )
            
        #-- END loop over mentions --#            
        reference_dict[ "mention_list" ] = mention_list        
        
        # contexts
        context_list = []
        context_qs = data_reference.datareferencecontext_set.all()
        for context in context_qs:
            
            # add value to list.
            context_list.append( context.value )
            
        #-- END loop over mentions --#            
        reference_dict[ "context_list" ] = context_list        
        
        # append reference to list.
        reference_list.append( reference_dict )
    
    #-- END loop over data references --#
    
#-- END loop over article data --#

# output to file.
json_file_path = "{}/data_references.json".format( output_folder_path )
with open( json_file_path, "w", encoding = "utf-8" ) as json_file:

    json.dump( reference_list, json_file, indent = 4, sort_keys = True )

#-- END with open( json_file_path, "w", encoding = "utf-8" ) as json_file: --

print( "total data reference count: {} (updated {})".format( data_reference_counter, update_counter ) )